In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.stat import Correlation
#import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
#import holidays
from datetime import datetime, timezone
from pyspark.ml.classification import RandomForestClassifier, BinaryLogisticRegressionSummary
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.pipeline import PipelineModel

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("US_Accidents") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR") #supress warnings

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/25 15:10:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/25 15:10:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/25 15:10:36 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/25 15:10:36 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/11/25 15:10:36 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/11/25 15:10:36 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [3]:
# Read in Data
df = spark.read.parquet("updated_dataset.parquet")

In [4]:
# Binary Classification (1 or 2 vs 3 or 4)
#df = df.withColumn('Severity_Binary', when((col("Severity")==1) | (col("Severity")==2), 0).otherwise(1))
df = df.drop('Severity')
df.show(5)

+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+
|Temperature|Humidity|Pressure|Visibility|Wind_Speed|Precipitation|Weekday|Rush_Hour|Rain|Snow|    SeasonVec|Astronomical_TwilightIndex|Interstate_Indicator|Sex_ratio|Percent_Age_65_over|MedianIncome|Urban_Ratio|Traffic_Interference|Traffic_Intersection|Destination|Percent_Age_15-24|Severity_Binary|
+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+
|         21|      85|      30|         1|        10|            0|      1|        1|   0|   1|(3

In [5]:
# Create list of features
feature_list = []
for col in df.columns:
    if col == 'Severity':
        continue
    elif col == 'Severity_Binary':
        continue
    else:
        feature_list.append(col)

In [6]:
# Split the data into train and test
splits = df.randomSplit([0.8, 0.2], 314)
train = splits[0]
test = splits[1]

In [7]:
from pyspark.sql import functions as F

# Step 1: Group by 'Severity' and count occurrences
class_counts = train.groupBy("Severity_Binary").count()

# Step 2: Use PySpark's max() function to find the maximum count
max_class_size = class_counts.agg(F.max('count')).collect()[0][0]

# Initialize a list to store the oversampled DataFrames for each class
oversampled_df_list = []

# Step 3: Oversample each class to match the maximum class size
for row in class_counts.collect():
    class_label = row['Severity_Binary']
    class_size = row['count']

    # Filter data for the current class
    class_data = train.filter(F.col("Severity_Binary") == class_label)

    if class_size < max_class_size:
        # Oversample the class to match the maximum class size
        class_data_oversampled = class_data.sample(withReplacement=True, fraction=max_class_size / class_size)
    else:
        # Retain the original data for the class if it's already at the maximum size
        class_data_oversampled = class_data

    # Add the oversampled data to the list
    oversampled_df_list.append(class_data_oversampled)

# Step 4: Combine all the oversampled DataFrames
oversampled_train = oversampled_df_list[0]  # start with the first one
for class_df in oversampled_df_list[1:]:
    oversampled_train = oversampled_train.union(class_df)

# Show the result of oversampling
oversampled_train.show()

# Step 5: Group by 'Severity' and count the occurrences in the oversampled DataFrame
oversampled_class_counts = oversampled_train.groupBy("Severity_Binary").count()

# Show the result
oversampled_class_counts.show()

+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+
|Temperature|Humidity|Pressure|Visibility|Wind_Speed|Precipitation|Weekday|Rush_Hour|Rain|Snow|    SeasonVec|Astronomical_TwilightIndex|Interstate_Indicator|Sex_ratio|Percent_Age_65_over|MedianIncome|Urban_Ratio|Traffic_Interference|Traffic_Intersection|Destination|Percent_Age_15-24|Severity_Binary|
+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+
|        -17|      80|      29|        10|         6|            0|      1|        0|   0|   0|(3

+---------------+-------+
|Severity_Binary|  count|
+---------------+-------+
|              1|4576169|
|              0|4579284|
+---------------+-------+



In [8]:
# Assemble data for logistic regression model
assembler = VectorAssembler(inputCols=feature_list,
                            outputCol="features")

oversampled_train = assembler.transform(oversampled_train)
test = assembler.transform(test)

In [9]:
# Standardize the predictors
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(oversampled_train)
scaledTrainData = scalerModel.transform(oversampled_train)
scaledTestData = scalerModel.transform(test)

In [10]:
# Fit logistic regression model with intercept
from pyspark.ml.classification import LogisticRegression

# instantiate the model
lr = LogisticRegression(labelCol='Severity_Binary',
                        featuresCol='scaledFeatures')

# Fit the model
lrModel = lr.fit(scaledTrainData)
print(f"Detected number of classes: {lrModel.numClasses}")

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

Detected number of classes: 2
Coefficients: DenseMatrix([[-0.15981222,  0.01118674,  0.15384289,  0.0445253 ,  0.14519159,
               0.00551605, -0.05349261, -0.00065568,  0.05476995, -0.013684  ,
              -0.33131213, -0.1869647 , -0.11208113, -0.04724808,  0.55570483,
               0.03441062, -0.10494079, -0.00975087,  0.0453488 ,  0.01478961,
              -0.19433603, -0.13347267, -0.03161235]])
Intercept: [-3.9793317859531854]


In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# compute predictions. this will append column "prediction" to dataframe
lrPred = lrModel.transform(scaledTestData)
lrPred.select("prediction").distinct().show()

# Assuming `lrPred` is your predictions DataFrame
evaluator = MulticlassClassificationEvaluator(
    labelCol="Severity_Binary",
    predictionCol="prediction",
    metricName='truePositiveRateByLabel', 
    metricLabel=1.0
)

recall = evaluator.evaluate(lrPred)
print(f"Recall for label 1.0: {recall}")

accuracy = evaluator.evaluate(lrPred, {evaluator.metricName: "accuracy"})
print(f'Accuracy: {accuracy}')

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
+----------+



Accuracy: 0.6865544449091897


In [12]:
precision = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedPrecision'})
recall = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedRecall'})
f1_score = evaluator.evaluate(lrPred, {evaluator.metricName: 'f1'})

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')

Precision: 0.7745788988352853
Recall: 0.6865544449091897
F1 Score: 0.7162489414442448


In [13]:
# Add a column to indicate correct or incorrect predictions
predictions = lrPred.withColumn(
    'is_correct', F.expr("CASE WHEN Severity_Binary = prediction THEN 1 ELSE 0 END")
)

# Calculate accuracy by class
accuracy_by_class = predictions.groupBy('Severity_Binary').agg(
    (F.sum('is_correct') / F.count('Severity_Binary')).alias('accuracy')
)

# Show per-class accuracy
accuracy_by_class.show()

+---------------+------------------+
|Severity_Binary|          accuracy|
+---------------+------------------+
|              1|0.5733423794300959|
|              0|0.7123517995426683|
+---------------+------------------+



In [15]:
# Calculate metrics by class
labels = [row['Severity_Binary'] for row in lrPred.select('Severity_Binary').distinct().orderBy('Severity_Binary').collect()]

metrics = {}
for label in labels:
    # Filter predictions for the current label
    true_positive = lrPred.filter((F.col('Severity_Binary') == label) & (F.col('prediction') == label)).count()
    false_positive = lrPred.filter((F.col('Severity_Binary') != label) & (F.col('prediction') == label)).count()
    false_negative = lrPred.filter((F.col('Severity_Binary') == label) & (F.col('prediction') != label)).count()
    true_negative = lrPred.filter((F.col('Severity_Binary') != label) & (F.col('prediction') != label)).count()

    # Precision, Recall, and F1 Score
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    # Store metrics
    metrics[label] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}

# Print metrics for each class
for label, metric in metrics.items():
    print(f"Class {label} - Precision: {metric['precision']:.4f}, Recall: {metric['recall']:.4f}, F1 Score: {metric['f1_score']:.4f}")

Class 0 - Precision: 0.8799, Recall: 0.7124, F1 Score: 0.7873
Class 1 - Precision: 0.3123, Recall: 0.5733, F1 Score: 0.4044
